In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
import peakutils
from peakutils.plot import plot as pplot
from matplotlib import pyplot

import talib

In [2]:
class CSVDataHandler:
    def __init__(self, csv_dir, symbol_list, timeframe):
        self.csv_dir = csv_dir
        self.symbol_list = symbol_list
        self.timeframe = timeframe
        
        self.symbol_data = dict()
        self.latest_symbol_data = dict()
        
        self.continue_backtest = True
        
        self._import_symbol_data()
        
    def _import_symbol_data(self):
        csv_files_path = f'{Path().absolute()}/{self.csv_dir}'
        columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        combined_symbol_index = None
        
        for symbol in symbol_list:
            symbol_csv_path = f'{csv_files_path}/{symbol}_{self.timeframe}.csv'
            self.symbol_data[symbol] = pd.read_csv(symbol_csv_path, header=None, index_col=0, names=columns)
            
            self.symbol_data[symbol].index = pd.to_datetime(self.symbol_data[symbol].index, unit='ms')
            self.symbol_data[symbol] = self.symbol_data[symbol].drop_duplicates()
            self.symbol_data[symbol] = self.symbol_data[symbol].sort_index()
            
            if combined_symbol_index is None:
                combined_symbol_index = self.symbol_data[symbol].index
            else:
                combined_symbol_index = combined_symbol_index.union(self.symbol_data[symbol].index)
            
            self.latest_symbol_data[symbol] = list()
            
        for symbol in symbol_list:
            self.symbol_data[symbol] = self.symbol_data[symbol].reindex(index=combined_symbol_index, method='pad', fill_value=0).itertuples()
            
    def _get_new_bar(self, symbol):
        for bar in self.symbol_data[symbol]:
            yield bar
            
    def update_bars(self):
        for symbol in self.symbol_list:
            try:
                bar = next(self._get_new_bar(symbol))
            except StopIteration:
                self.continue_backtest = False
            else:
                if bar is not None:
                    self.latest_symbol_data[symbol].append(bar)
                    
    def get_latest_bar(self, symbol):
        try:
            return self.latest_symbol_data[symbol][-1]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars(self, symbol, N=1):
        try:
            return np.array(self.latest_symbol_data[symbol][-N:])
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bar_datetime(self, symbol):
        try:
            return self.latest_symbol_data[symbol][-1].Index
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bar_value(self, symbol, value_type):
        try:
            return getattr(self.latest_symbol_data[symbol][-1], value_type)
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars_values(self, symbol, value_type, N=1):
        try:
            bars = self.latest_symbol_data[symbol][-N:]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
        else:
            return np.array([getattr(bar, value_type) for bar in bars])

In [287]:
symbol_list = ['BTC-USDT']
data_handler = CSVDataHandler('../exchange_data', symbol_list, timeframe='1h')

In [298]:
data_points = 3360
for i in range(3360):
    data_handler.update_bars() 

In [324]:
datetimes = data_handler.get_latest_bars_values('BTC-USDT', 'Index', N=data_points)
closes = data_handler.get_latest_bars_values('BTC-USDT', 'close', N=data_points)
data = pd.DataFrame(data=closes, index=datetimes, columns=['close'])
close_weekly = data.resample('W').agg({
'close': lambda x: x[-1]
})['close'].to_numpy()

In [3]:
columns = {0: 'datetime', 1: 'open', 2: 'high', 3: 'low', 4: 'close', 5: 'volume'}
btc = pd.read_json('../exchange_data/BTC-USDT_15m.json')

In [4]:
btc = btc.rename(columns=columns)

In [5]:
btc.to_csv('BTC-USDT_15m.csv', index=False, header=False)